# 2021학년도 교과목 유사도 분석

In [270]:
# Library Import
import re
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

# Tokenize & Vectorize
from ckonlpy.tag import Twitter
import gensim
from gensim.models import Word2Vec

# 한글 형태소 분석
from eunjeon import Mecab
mecab = Mecab()

# 영어 형태소 분석
import nltk
from nltk import word_tokenize

# Visualize
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from tqdm import tqdm

In [271]:
# 파일 호출: 강의자료
df_sbjt = pd.read_excel('2021/강의자료(20210930).xlsx', engine='openpyxl')

In [272]:
# 파일 호출: 주별 강의일정
df_week = pd.read_excel('2021/강의주차별자료(20210930).xlsx', engine='openpyxl')

In [273]:
# 학부 과목만 추출
df_sbjt = df_sbjt.loc[df_sbjt['UNIV_GRSC_FG'] == '대학']
df_week = df_week.loc[df_week['UNIV_GRSC_FG'] == '대학']

In [274]:
# 계절학기 과목은 제외
df_sbjt = df_sbjt.loc[df_sbjt['SHTM_NM'].isin(['1학기', '2학기'])]
df_week = df_week.loc[df_week['SHTM_NM'].isin(['1학기', '2학기'])]

In [275]:
# 분반 정렬
df_sbjt.sort_values(['CLSS_NO'], ascending=True, inplace=True)
df_week.sort_values(['CLSS_NO'], ascending=True, inplace=True)

In [276]:
# 학부번호 unique한 값으로 drop: keep='first'로
df_sbjt.drop_duplicates(['HAKSU_NO'], keep='first', inplace=True)
df_week.drop_duplicates(['HAKSU_NO'], keep='first', inplace=True)

In [277]:
# 필요한 열만 추출
df_sbjt_tmp = df_sbjt[['UNIV_GRSC_FG', 'LT_YY', 'SHTM_NM', 'HAKSU_NO', 'SBJT_KOR_NM', 'STAFF_NO', 'STAFF_NM', 'DEPT_NM', 'LT_TYPE_CD', 'OPEN_COLG', 'OPEN_SUST', 'SBJT_ALL']]

In [278]:
df_week_tmp = df_week[['HAKSU_NO', 'WEKALL']]

In [279]:
df_merge = pd.merge(df_sbjt_tmp, df_week_tmp, on='HAKSU_NO')

In [280]:
df_merge.to_excel('2021학년도 학부 교과목 tmp.xlsx')

## 전처리

In [281]:
# 추출 파일 로드
# df = pd.read_excel('2021/2021학년도 학부 교과목 추출 완료.xlsx', engine='openpyxl')

In [282]:
# 추출 파일 로드
df_2 = pd.read_excel('2021/2021학년도 학부 교과목 추출 완료(행정부속기관제외).xlsx', engine='openpyxl')

In [283]:
df_new = df.drop(['SBJT_ALL', 'WEKALL'], axis=1)

In [284]:
# df_new = df_2.drop(['SBJT_ALL', 'WEKALL'], axis=1)

In [285]:
# 특수문자 처리
df_new['ALL'] = df_new['ALL'].str.replace(pat=r'[-=,/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》■□●○◆①②③④【】▶]', repl=r' ', regex=True)

In [286]:
# 줄바꿈 문자 처리
df_new['ALL'] = df_new['ALL'].str.replace("\r", "")
df_new['ALL'] = df_new['ALL'].str.replace("\n", "")
df_new['ALL'] = df_new['ALL'].str.replace("\t", "")
df_new['ALL'] = df_new['ALL'].str.replace("\r\n", "")
df_new['ALL'] = df_new['ALL'].str.replace("_x000D_", "")
df_new['ALL'] = df_new['ALL'].str.replace("_x000D_2", "")
df_new['ALL'] = df_new['ALL'].str.replace("_x000D_3", "")
df_new['ALL'] = df_new['ALL'].str.replace("_x000D_4", "")

In [292]:
# twitter 사용을 위한 경로추가
import sys
sys.path.insert(0, '../')
import ckonlpy
print('ckonlpy version = {}'.format(ckonlpy.__version__))

ckonlpy version = 0.0.64


In [293]:
# 사용자 정의 사전 추가
twitter = Twitter(use_twitter_dictionary=False)

twitter.add_dictionary(['자율사물', '통신기기', '데이터사이언스', '드론', '사물인터넷', '캡스톤', '머신러닝','시각화','c언어','r프로그래밍','maxwell', '대면', '비대면', '플립드러닝', '인공지능', '블록체인', '데이터분석', '유전자분석', '진로설정', '유전자가위', '데이터경제', '수익개선모델', '기술기반', '3d프린터','자율주행', '인문소양', '문제해결', '공학적', '재정의', '아이디에이션', '시제품', '데이터사이언스', '데이터활용', '다용도', '캡스톤디자인', '캡스톤 디자인','분석기법', '자기주도', '문제해결능력', '사회문제', '4차산업혁명', '산업혁명시대', '성공', '전인미답', '청년기업가', '산업혁명', '불확실성', '디자인싱킹', '디자인씽킹', '지도교수', '기업가정신', '시사점', '문제발굴', '현장실습', '비주얼', '인터랙션', '비전공', '공학', '설계', '문제정의', '절차별', '영상처리', '화질개선', '영상복원', '해결방안', '로지스틱', '의사결정나무', '상호작용', '적정법', '전반부', '후반부', '단위세포', '비가역적', '완충작용', '민간부문', '기초통계학', '통계자료', '기초통계', '포아송분포', '여사건', '이항실험', '교수자', '크리스퍼', '정밀의료', '의료산업','입문과정', '베이즈', '베이즈정리', '상관계수', '도수분포표', '꺾은선그래프', '이산확률분포', '카이제곱분포', '구간추정', '인과관계', '상관관계', '해석기하', '여러 가지', '코시정리', '복소함수', '실함수', '고교수학', '항등정리', '푸비니 정리', '기초지식', '크라메르', '문제풀이', '마르코프', '모형화', '초기하분포', '분포함수', '비누화', '고체상', '상변화', '결합이론', '분자구조', '주기성', '전자구조', '화학정원', '멘델유전법칙', '유전법칙', '복합형질', '유전현상', 'dna', 'rna', 'mrna', '기본단위', '소기관', '막수송', '신호전달', '생명공학', '인간세포', '유전공학', '조직공학', '단백질공학', '기초의과학', '줄기세포학', '생체의학공학', '재조합', '생의학', '바이오센서', '동물세포', '식물세포', 'matlab', '건축공학', '개념설계','상세설계', '분석기법', '문화인류학', '최신기술', '딥러닝', '데이터과학', '공공데이터','데이터세트', 'ict', 'iot', '개선안', '행정문제', '분과학문', '정책보고서', 'r', 'sas', 'python', '데이터마이닝', 'cad', '평판계수법', '비성장속도', '현대사회', '기초개념', '현대사', '신자유주의', '세계화', '환경위기', 't-test', '텍스트마이닝', '비정형데이터', '기술통계량', '사회구조', '조직원리', '구조변동', '한국사회', '집합행동','사회운동', '사회적자아', '금융화', ' 위험사회', '근대사회', '사회현상', '사회이동', '현대가족', '로컬푸드', '사회적기업','등한시','성장지상주의', '순기능','역기능','취업난','대응방식', '분석법', '직업윤리', '요구조건', '설계이론', '사회상', '사회복지', '다문화사회', '다문화현상', '21세기', '20세기', '19세기','니즈다오', '양가불이', '적용범위', '도구', '이진트리', '트리구조', '우선순위', '해시함수', '자연어처리', '인공신경망', '랜덤포레스트', '서포트', '벡터', '머신', '차원축소', '비지도학습', '결정트리', 'tensorflow', 'keras', '이진탐색', '이진탐색나무', '선택정렬','버블정렬','삽입정렬','쉘정렬','퀵정렬','합병정렬','힙정렬','계수정렬','기수정렬','버킷정렬','외부정렬','합병정렬', '비지도', '경진대회', '군집분석', '질의응답', '사업계획서', '음향인식', '이미지인식', '동작인식', '생체신호', '기술창업','연구결과', '20세기', '어디서든', '방재계획', '기초학문', '공간정보시스템', '취약지역', '지리정보','진로개발', '미래지향','교과과정','키워드', '대북외교', '현장방문', '전문지식', '미세플라스틱','배리커머너','에스노그래피', '연계', '정보통신', '정보통신기술','상호관계', '거시적', '사회이슈', '지혜', '테크노사이언스', '여성운동', '포텐셜에너지', '회전운동', '토크', 'sw설계', 'sw', 'ux', 'hci','상용성','사회철학', '자유주의', '실증주의', '산물', '법치주의', '봉건체제', '계몽주의', '근대', '수리모형','스프레드시트', '경영과학', '유효성', '실세계', 'gps', '지오레퍼런싱', '좌표체계', '원격탐측','다변화', 'gis', '중앙아시아', '방재학', '화공생물공학', '델타함수', 'green','gauss','stoke', '비제차', 'laplace', '충격함수', '벡터장', '흐름선', '전기적', '벡터함수','벡터미적분', 'bernoulli', 'riccati','heaviside', 'frobenis','dirac', 'leibniz', '풀이법', 'r프로그램', '산술기하', '일차합동식', '쌍곡선','다변수함수', 'taylor', 'maclaurin', '분자운동', 'kinetic','kinetics','호제법', 'hare', '벡터공간', 'cramer','jordan','n차원', '대각화','여인수','연립', 'c++', 'c프로그래밍', 'FOR문', 'for문', 'while문', 'html', 'css', 'newton', 'java', '모평균', '모비율', '아이프레임', '제공자', '산업공학', '제어문', '딕셔너리', 'labview', '미로찾기', '실습', '확산방정식', 'machine' 'learning', '지배방정식', '상미분', '급수해', '미세조직', 'hess', '분자수준', '분포형태', '자료분석', '클라우드', '컴퓨팅', '생성주기', '빅브라더', '마이닝', '웹상', 'sns', '감성분석', '크롤링','c#', 'pbl'], 'Noun')

C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [294]:
# 한글 불용어 정의
stopwords = ['은', '는', '이', '가', '을', '를', '의', '이', '들', '좀', '잘', '과', '도', '으로', '자', '에', '와', '한', '하다', ',', '.', '및',  '그리고', '그래서', '하지만', '그러므로', '중간고사', '기말고사', 'webex', '미팅번호', '본', '강좌', '과목', '강의', '대면강의', '전용강의', '전용강좌', '절대평가', '코로나19', '코로나', '있다', '적', '인', '대해', '이고', '하고', '에서', '되고', '으로의','또한', '로', '등', '이다', '대한', '별', '여러', '되다', '학습', '익히다', '습득', '에서의', '이해', '알', '수', '다루다', '기르다', '지식', '통해', '통한', '부', 'ㆍ', '장', '중간', '기말', '시험', '성적', '평가', '방법', '이란', '개요', '소개', '내용', '반기', '오리엔테이션', '무엇', '인가', '보다', '에서는', '여러가지', '가장', '으로써', '으로서', '특히', '에는', '주다', '배우다', '에게', '이자', '이나', '바탕', '함께', '대학', '관련', '학문', '학부', '누가', '교과목', '수업', '비대면', '비대', '대면', '분야', '이기', '그간', '위해', '과제', '제출', '최근', '우리', '이라는', '학년', '학생', '대상', '본격', '이수', '전공', '관심', '동기', '부여', '학점', '학과', '입학', '신입생', '로서', '로써', '거나', '이거나', '되어다', '이외', '수시로', '양해', '제대로', '까지', '에서도', '이며', '많이', '발표', '토론', '퀴즈', '팀별', '으로부터', '않다', '모든', '그러나', '그래서','따라서','그러므로', '이지만', '이의', '주차', '보강', '첫째', '둘째', '셋째', '넷째', '실시간', '클래스', '대하', '화하다', '배우다', '돕다', '으로는', '일상생활', '쉽게', '수강', '권장', '학기', '플립드', '수강생', '토대', '이처럼', '구체', '각각', '거듭', '궁극', '나은', '직접','본인', '급변', '일련', '단순', '때문', '팀 편성', '팀편성', '전반부', '후반부', '먼저', '공부', '더욱', '어디', '하든', '스스로', '기반', '이용', '로부터','단지','교수자', '마치', '비롯', '장차', '도모', '누구', '주변', '최소한', '배양', '함양', '능력', '발전', '증진', '목표', '주요', '의미', '여러 가지', '고교수학', '관해', '별로','첫걸음','교량','역할','대비', '여러', '가지', '고사', '조편성', '보충', '생활','전반','점차','증대','달리', '향후',  '반드시','숙지', '널리', '간의', '마음대로', '만들기', '개인', '위함','여러분','모두', '강의계획서', '참조', '별첨', '동시', '좀더', '개괄', '깊이', '만큼', '각종', '여기', '학습자', '대부분', '차지', '초점', '이후','다른', '공지', '예정', '질의응답', '어서', '그것','매우', '곳곳', '언제', '어디서든','리뷰','교과과정','거기','역시', '두루', '오해', '강사', '난이도','조가','내외','삼고','주요한','측면','취하다', '문제','당면', '미션', '인터뷰','다양','형태','실상', '지니', '매일','오늘','다음','가운데','바로','교재','새로이','경우','부터','동안','누군가','온갖','지난','혹시','졸업','제공','서도','갖축','기틀','마련', '둘러보기', '한글날', '추석', '설날', '연휴', '어로', '거의', '이번', '더욱이', '주로', '앞서', '두번째', '보고', '지금', '현재', '복습', '겸비', '구분', '오직', '자체', '하나', '당시', '크게', '계속', '대학생','적극', '오늘날', '갈수록', '근간', '교육', '#NAME?']

In [295]:
# # 영어 불용어 처리
# nltk.download('stopwords')
STOP_WORDS = nltk.corpus.stopwords.words('english')

In [296]:
# 문장을 형태소 단위로 토큰화하는 함수 생성
def tokenizing(concat_str):
    
    temp_X = twitter.pos(concat_str, stem=True, norm=True)                  # 단어 토큰화: 품사도 함께 표시되도록

    temp_X = [word[0] for word in temp_X if word[1] in ["Noun", "Alpha"]]   # 명사와 영어만 저장
    temp_X = [word for word in temp_X if not word in stopwords]             # 한글 불용어 제거
    temp_X = [word for word in temp_X if not word in STOP_WORDS]            # 영어 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]                     # 한글자짜리 형태소는 제거
    
    return temp_X

In [297]:
# 토큰화 함수 적용 후, 결과를 데이터프레임 열에 넣는다.
df_new['TK_ALL'] = df_new['ALL'].apply(lambda x: tokenizing(x))

In [298]:
# 영어를 소문자로 바꿔주기
for i in range(len(df_new)):
    for j in range(len(df_new['TK_ALL'][i])):
        df_new['TK_ALL'][i][j] = df_new['TK_ALL'][i][j].lower()

In [299]:
# 표제어 추출 : 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿈
from nltk.stem import WordNetLemmatizer
df_new['TK_ALL'] = df_new['TK_ALL'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='n') for word in x])

In [300]:
# 토큰화 된 데이터를 result 리스트에 때려넣기
result = []
append = result.append

for i in range(len(df_new['TK_ALL'])):
    append(df_new['TK_ALL'].values[i])

In [302]:
# 모델호출 & 훈련
model = gensim.models.Word2Vec.load('word2vec')
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [303]:
total_examples = model.corpus_count
print(total_examples)

2422


In [304]:
# 트레이닝
model.train(result, total_examples=total_examples, epochs=10)

(3221892, 3657940)

In [305]:
# model로부터 단어벡터 구하기
word_vectors = model.wv

In [306]:
vocabs = list(word_vectors.vocab.keys())              # 단어
word_vectors_list = [word_vectors[v] for v in vocabs] # 벡터값

In [307]:
# 딕셔너리 형태로 만들기: {'단어' : 벡터값}
vocab_vector_dict = dict(zip(vocabs, word_vectors_list))

In [308]:
len(vocab_vector_dict)

8128

In [309]:
# 단어 벡터의 평균 구하기
def get_avg(TK_ALL):
    base_vector = np.zeros(shape = (100,))
    cnt = 0
    for token in TK_ALL:
        try:
            base_vector += vocab_vector_dict[token]
            cnt += 1
        except KeyError:
            continue
    return base_vector/cnt

In [310]:
# 함수 적용
df_new['ALL_vector'] = df_new['TK_ALL'].apply(lambda x: get_avg(x))

In [311]:
# 기존 인덱스 번호로 되어있는 것을 리셋
df_new.reset_index(drop=True, inplace=True)

In [312]:
# 코사인 유사도 구하는 함수 생성
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [313]:
# 최종 데이터프레임
final_df = pd.DataFrame(columns=['년도', '학기', '과목A', '학수번호', '단과대', '학부/학과', '교수', '개인번호', 'YEAR', 'SHTM_NM', '과목B', 'HAKSU_NO', 'OPEN_COLG', 'OPEN_SUST', 'STAFF_NM', 'STAFF_NO', 'TK_ALL', '유사도'])

In [314]:
for n in tqdm(range(len(df_new))):
    # 비교하고자 하는 교과목의 벡터값
    all_query_vector = df_new['ALL_vector'].values[n]

    # 같은 '학부/학과'의 concat_vector와는 유사도를 계산하지 않는다.
    all_cos_list = []
    append = all_cos_list.append
    for ALL_vector in (df_new['ALL_vector'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values:
        append(cos_sim(all_query_vector, ALL_vector))

    # 결과 DataFrame 생성
    result_df = pd.DataFrame(columns=['년도', '학기', '과목A', '학수번호', '단과대', '학부/학과', '교수', '개인번호', 'YEAR', 'SHTM_NM', '과목B', 'HAKSU_NO', 'OPEN_COLG', 'OPEN_SUST', 'STAFF_NM', 'STAFF_NO', '유사도'])

    # 결과로 도출된 과목
    result_df['YEAR'] = (df_new['LT_YY'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['SHTM_NM'] = (df_new['SHTM_NM'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['과목B'] = (df_new['SBJT_KOR_NM'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['HAKSU_NO'] = (df_new['HAKSU_NO'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['OPEN_COLG'] = (df_new['OPEN_COLG'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['OPEN_SUST'] = (df_new['OPEN_SUST'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['STAFF_NM'] = (df_new['STAFF_NM'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['STAFF_NO'] = (df_new['STAFF_NO'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['TK_ALL'] = (df_new['TK_ALL'][df_new['OPEN_SUST'] != df_new.iloc[n]['OPEN_SUST']]).values
    result_df['유사도'] = all_cos_list

    # 유사도 값 채우기
    result_df = result_df[np.isfinite(result_df['유사도'])]
    result_df = result_df.sort_values(by=['유사도'], ascending=False).reset_index(drop=True)

    # 비교하고자 하는 과목의 정보
    result_df['년도'] = df_new['LT_YY'].values[n]
    result_df['학기'] = df_new['SHTM_NM'].values[n]
    result_df['과목A'] = df_new['SBJT_KOR_NM'].values[n]
    result_df['학수번호'] = df_new['HAKSU_NO'].values[n]
    result_df['단과대'] = df_new['OPEN_COLG'].values[n]
    result_df['학부/학과'] = df_new['OPEN_SUST'].values[n]
    result_df['교수'] = df_new['STAFF_NM'].values[n]
    result_df['개인번호'] = df_new['STAFF_NO'].values[n]

    # 유사도 0.90 이상의 과목만 보여준다.
    top_df = result_df.loc[result_df['유사도'] >= 0.90]

    # 생성된 데이터프레임을 합친다.
    final_df = pd.concat([final_df, top_df])

100%|██████████| 2422/2422 [01:42<00:00, 23.74it/s]


In [315]:
# 유사도 높은 순으로 정렬
final_df.sort_values(['유사도'], ascending=False, inplace=True)

In [316]:
# 기존 인덱스 번호로 되어있는 것을 리셋
final_df.reset_index(drop=True, inplace=True)

In [317]:
len(final_df)

44392

In [ ]:
final_df.loc[final_df['유사도'] >= 0.95].head(20)

In [320]:
# 저장
# final_df.to_csv('w2v_0930_2.csv', encoding='cp949')